In [ ]:
import diplib as dip
import numpy as np
import matplotlib.pyplot as plt
import os
from utilities import *

In [ ]:
# dip_blue, blue_names = load_dip_images('blues')
# # dip_gray, grays_names = load_dip_images('gray')
# dip_blue[21].Show()

In [ ]:
# create blu and gray images from data
dip_images, dip_names = load_dip_images('data')

dip_gray, dip_blue = make_grayscale(dip_images)
save_images(dip_gray, 'gray', "gray", file_type=["tif"])
save_images(dip_blue, 'blues', "blue", file_type=["tif"])

dip_gray[21].Show()

In [ ]:
# threshold and morphological operators
dip_thresh = threshold_images(dip_blue)
save_images(dip_thresh, 'thresh', "thresh", file_type=["tif"])
dip_transf = apply_transformations(dip_thresh)
save_images(dip_transf, 'transf', "transf", file_type=["tif"])

dip_thresh[21].Show()
dip_transf[21].Show()

In [ ]:
features = ['Perimeter', 'Size',
                'Circularity','Roundness',
                'Statistics']
                
dip_labeled, measurements = measure_elements(dip_transf, dip_gray, features=features)

In [ ]:
measurements

In [ ]:
img = dip_images[0]
arr = np.array(img)
aa = arr[:,:,2]
blue = dip.Image(aa)
blue.Show()
blue = dip.Gauss(blue, sigmas=[2])
thresh= dip.TriangleThreshold(blue)

open = dip.Opening(thresh)


thresh.Show()

In [ ]:
arr

In [ ]:
np.array(blue)

In [ ]:
np.where(aa ==0)

In [ ]:
np.array(dip_images[:,;,2])

In [ ]:
blues=[]
grays = []
for img in dip_images:
    array= np.array(img)
    new_img = dip.ColorSpaceManager.Convert(img, "gray")
    blue_img = dip.Image(array[:,:,2])
    blues.append(blue_img)
    grays.append(new_img)